In [15]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV



In [16]:
PCA = False
if PCA:
    X = pd.read_csv("msia420PA_project/Dimension Reduction/PCA_dat.csv", index_col=0) # PCA Model
else:
    X = pd.read_csv("full_dat.csv", index_col=0) # full model


y = pd.read_csv("msia420PA_project/Hotel Reservations.csv")['booking_status']

le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=None, train_size=0.3)

In [ ]:
parameters = {
    'learning_rate': [0.001, 0.01, 0.05, 0.1, None],
    'max_depth': [2,5,7,10,None]
}


xgbr = xgb.XGBClassifier()
clf = RandomizedSearchCV(estimator=xgbr,
                         param_distributions = parameters,
                         n_iter=20,
                         verbose=1,
                         cv = 5,
                         scoring = 'roc_auc')
clf_xgb = clf.fit(X_train, y_train)
print("Best parameters:", clf.best_params_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
from sklearn.metrics import roc_auc_score

y_pred = clf_xgb.predict(X_test)
classifier_score_xgb = roc_auc_score(y_test, y_pred)
classifier_score_xgb